In [1]:
import openai
from dotenv import dotenv_values
config = dotenv_values(".env")
openai.api_key = config["OPENAI_API_KEY"]

In [2]:
import pandas as pd
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt
import pickle
import tiktoken
from nomic import atlas

In [3]:
dataset_path = "./movie_plots.csv"
dataframe = pd.read_csv(dataset_path)

In [4]:
# Narrow our data set to 5000 recent American movies (to save money):
movies = dataframe[dataframe["Origin/Ethnicity"] == "American"].sort_values("Release Year", ascending=False).head(5000)

In [5]:
# Extract the movie plots into a list:
movie_plots = movies["Plot"].values  # ["plot1", "plot2", ...] 

In [6]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-ada-002"):

    # replace newlines, which can negatively affect performance:
    text = text.replace("\n", " ")

    return openai.Embedding.create(input=text, model=model)["data"][0]["embedding"]

In [7]:
# Use the tiktoken library to encode a text using the "text-embedding-ada-002" model
enc = tiktoken.encoding_for_model("text-embedding-ada-002")

In [8]:
# calculate total tokens to estimate cost
total_tokens = sum([len(enc.encode(plot)) for plot in movie_plots]) # sum([697, 757, 361, ...])

In [9]:
total_tokens
cost = (.0004 / 1000) * total_tokens
print(f"Estimated cost is ${cost:.2f}")

Estimated cost is $1.45


In [10]:
# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embedding cache
embedding_cache_path = "movie_embeddings.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, or otherwise request via the API
def embedding_from_string(
    string,
    model="text-embedding-ada-002",
    embedding_cache=embedding_cache
):
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        print(f"GOT EMBEDDING FROM OPENAI FOR '{string[:20]}'")
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

In [23]:
plot_embeddings = [embedding_from_string(plot, model="text-embedding-ada-002") for plot in movie_plots]
# len(plot_embeddings) # 5000 
# len(plot_embeddings[0]) # 1536

In [24]:
data = movies[["Title", "Genre"]].to_dict("records")  # list of dicts: [{'Title': 'Phantom Thread', 'Genre': 'drama'}, ...]

In [25]:
project = atlas.map_embeddings(
    embeddings=np.array(plot_embeddings), 
    data = data
)

2023-06-01 20:44:36.414 | INFO     | nomic.project:_create_project:965 - Creating project `threatening-lashes` in organization `qmeng222`
2023-06-01 20:44:38.787 | INFO     | nomic.atlas:map_embeddings:100 - Uploading embeddings to Atlas.
4it [00:12,  3.21s/it]                             
2023-06-01 20:44:51.733 | INFO     | nomic.project:_add_data:1577 - Upload succeeded.
2023-06-01 20:44:51.736 | INFO     | nomic.atlas:map_embeddings:119 - Embedding upload succeeded.
2023-06-01 20:44:53.783 | INFO     | nomic.project:create_index:1282 - Created map `threatening-lashes` in project `threatening-lashes`: https://atlas.nomic.ai/map/a7eb77dd-03de-43a2-b6ac-c7bfe6e93e6c/228d53e7-4614-4796-802a-e74278ad7ae2
2023-06-01 20:44:53.784 | INFO     | nomic.atlas:map_embeddings:132 - threatening-lashes: https://atlas.nomic.ai/map/a7eb77dd-03de-43a2-b6ac-c7bfe6e93e6c/228d53e7-4614-4796-802a-e74278ad7ae2
